> 8. CIFAR10 이미지 데이터셋에 심층 신경망을 훈련해보세요. \
> a. 100개의 뉴런을 가진 은닉층 20개로 심층 신경망을 만들어보세요. He 초기화와 ELU 활성화 함수를 사용하세요.

In [2]:
from keras import datasets

(X_train_full, y_train_full), (X_test, y_test) = datasets.cifar10.load_data()

X_train = X_train_full[5000:]
y_train = y_train_full[5000:]
X_valid = X_train_full[:5000]
y_valid = y_train_full[:5000]

170498071/170498071 [==============================] - 343s 2us/step


In [3]:
X_train.shape

(45000, 32, 32, 3)

In [28]:
import keras 

model=keras.models.Sequential([
    keras.layers.Flatten(input_shape=[X_train.shape[1],X_train.shape[2],X_train.shape[3]]),
])

for _ in range(20):
    model.add(keras.layers.Dense(100, activation="elu",
                                 kernel_initializer="he_normal"))

> b. Nadam 옵티마이저, 조기 종료 사용한 네트워크 훈련 \
keras.datasets.cifar10.load_data()를 사용하여 데이터 적재가능   
10개 클래스, 32x32 크기, 컬러 이미지 60,000개 구성 \
10개 뉴런, 소프트 맥스 활성화 함수 출력층 필요.  \
적절한 학습률 ..

In [13]:
def recall(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Negative) = 실제 값이 1(Positive) 전체
    count_true_positive_false_negative = K.sum(y_target_yn)

    # Recall =  (True Positive) / (True Positive + False Negative)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())

    # return a single tensor value
    return recall


def precision(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Positive) = 예측 값이 1(Positive) 전체
    count_true_positive_false_positive = K.sum(y_pred_yn)

    # Precision = (True Positive) / (True Positive + False Positive)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    precision = count_true_positive / (count_true_positive_false_positive + K.epsilon())
    return precision

def f1score(y_target, y_pred):
    _recall = recall(y_target, y_pred)
    _precision = precision(y_target, y_pred)
    _f1score = ( 2 * _recall * _precision) / (_recall + _precision+ K.epsilon())
    return _f1score

In [29]:
model.add(keras.layers.Dense(10, activation="softmax"))

In [30]:
optimizer=keras.optimizers.Nadam(learning_rate=5e-5)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy",f1score])

In [31]:
import os

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_model.h5", save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

In [16]:
!pip install tensorboard

In [18]:
%load_ext tensorboard

In [19]:
%tensorboard --logdir=./my_cifar10_logs --port=6006

In [25]:
K = keras.backend

In [32]:
# X_means = X_train.mean(axis=0)
# X_stds = X_train.std(axis=0)
# X_train_scaled = (X_train - X_means) / X_stds
# X_valid_scaled = (X_valid - X_means) / X_stds
# X_test_scaled = (X_test - X_means) / X_stds

model.fit(X_train, y_train, epochs=100,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)

Epoch 1/100
1407/1407 [==============================] - 28s 15ms/step - loss: 4.1481 - accuracy: 0.1752 - f1score: 0.2633 - val_loss: 2.1532 - val_accuracy: 0.2204 - val_f1score: 0.1592
Epoch 2/100
1407/1407 [==============================] - 20s 14ms/step - loss: 2.0392 - accuracy: 0.2520 - f1score: 0.1345 - val_loss: 2.0920 - val_accuracy: 0.2470 - val_f1score: 0.2974
Epoch 3/100
1407/1407 [==============================] - 18s 13ms/step - loss: 1.9202 - accuracy: 0.3010 - f1score: 0.1817 - val_loss: 1.9050 - val_accuracy: 0.3080 - val_f1score: 0.2389
Epoch 4/100
1407/1407 [==============================] - 17s 12ms/step - loss: 1.8505 - accuracy: 0.3290 - f1score: 0.2301 - val_loss: 1.8731 - val_accuracy: 0.3222 - val_f1score: 0.3340
Epoch 5/100
1407/1407 [==============================] - 17s 12ms/step - loss: 1.7947 - accuracy: 0.3464 - f1score: 0.2751 - val_loss: 1.8017 - val_accuracy: 0.3508 - val_f1score: 0.3208
Epoch 6/100
1407/1407 [==============================] - 17s 12ms

In [33]:
model = keras.models.load_model("my_cifar10_model.h5")
model.evaluate(X_valid, y_valid)

ValueError: Unknown metric function: 'f1score'. Please ensure you are using a `keras.utils.custom_object_scope` and that this object is included in the scope. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

In [34]:
%tensorboard --logdir=./my_cifar10_logs --port=6006

Reusing TensorBoard on port 6006 (pid 5286), started 0:45:25 ago. (Use '!kill 5286' to kill it.)